<a href="https://colab.research.google.com/github/B-MEbrahim/Torch/blob/main/Basics/Build_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)

cuda


## Define the Class
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every nn.Module subclass implements the operations on input data in the forward method.

In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [11]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten
We initialize the `nn.Flatten` layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 3.1577e-01, -3.8848e-01,  7.2330e-01, -1.8457e-01,  1.6357e-01,
          3.1128e-02,  2.3788e-01,  1.7980e-01,  8.5956e-01, -3.3675e-01,
         -3.3157e-02,  4.1137e-01,  1.5647e-01, -2.4287e-01, -5.2708e-02,
          3.2463e-02, -8.9179e-01,  2.5170e-01,  6.6499e-02, -2.7762e-01],
        [ 3.4731e-01, -4.7633e-01,  3.6825e-01, -1.6155e-01,  4.2808e-03,
         -4.6696e-01,  4.6576e-01,  3.4283e-01,  8.9002e-01, -2.2844e-01,
         -2.4064e-02,  5.8981e-01,  4.3245e-01, -2.4301e-01, -4.1288e-01,
         -1.8908e-01, -6.6863e-01,  6.2131e-01,  3.7990e-01, -5.8007e-01],
        [ 4.3536e-01,  5.1392e-02,  3.4847e-01,  1.5894e-02,  1.7436e-01,
         -1.7670e-04,  6.3864e-01,  4.4419e-01,  1.0153e+00, -5.4052e-01,
          2.1260e-02,  7.9421e-01,  6.1227e-01, -2.5142e-01, -6.3093e-02,
         -2.3532e-01, -6.1501e-01,  7.8468e-01,  1.7236e-01, -4.0878e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3158, 0.0000, 0.7233, 0.0000, 0.1636,

## nn.Sequential
`nn.Sequential` is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. `dim` parameter indicates the dimension along which the values must sum to 1.

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0177, -0.0266, -0.0040,  ...,  0.0125,  0.0168,  0.0335],
        [ 0.0078,  0.0188, -0.0111,  ..., -0.0077, -0.0349,  0.0138]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0188,  0.0137], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0181, -0.0006, -0.0229,  ...,  0.0007, -0.0264, -0.0109],
        [ 0.0127,  0.0233, -0.0421,  ..., -0.0334, -0.0014,  0.0148]],
       device='cuda:0', grad_fn=<Sl